## 1. Importar librerías necesarias
Incluye librerías para manejo de archivos, procesamiento de texto, análisis de datos y embeddings.

## 1. Importar librerías necesarias
Incluye librerías para manejo de datos, procesamiento de texto y embeddings.

In [1]:
import os
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns

c:\Users\diego\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Lectura y limpieza de datos
Se leen los archivos *_Merged.csv de cada carrera y se construye el corpus por carrera.

In [3]:
ruta_base = 'todas_las_plataformas'
columnas_texto = ['skills', 'description', 'description_final', 'initial_skills', 'EURACE_skills']
corpus_carreras = {}
for subcarpeta in os.listdir(ruta_base):
    ruta_subcarpeta = os.path.join(ruta_base, subcarpeta)
    archivo_objetivo = os.path.join(ruta_subcarpeta, f'{subcarpeta}_Merged.csv')
    if os.path.isfile(archivo_objetivo):
        try:
            df = pd.read_csv(archivo_objetivo, dtype=str)
        except Exception as e:
            print(f'Error leyendo {archivo_objetivo}: {e}')
            continue
        textos = []
        for col in columnas_texto:
            if col in df.columns:
                textos.extend(df[col].dropna().tolist())
        corpus_carreras[subcarpeta] = ' '.join(textos)
df_ejemplo = pd.DataFrame({'carrera': list(corpus_carreras.keys()), 'texto': list(corpus_carreras.values())})
df_ejemplo

,carrera,texto
0,Administración_de_Empresas,"Servicio al cliente, Ventas, Gestión de efecti..."
1,Agroindustria,Cualquier desviación debe comunicarse al super...
2,Ciencia_de_Datos,Más de 3 años de experiencia en desarrollo de ...
3,Computación,"En general, estamos buscando un miembro del eq..."
4,Economía,"Sobresalir, Inteligencia de negocios, Power Bi..."
5,Electricidad,2 anos de experiencia minima como montador (pr...
6,Electrónica_y_Automatización,El candidato ideal tendrá experiencia en trata...
7,Física,Licenciatura y mínimo 9 años de experiencia re...
8,Geología,Grado avanzado (maestría o doctorado) en geolo...
9,Ingeniería_Ambiental,"Una licenciatura en un campo científico, de in..."


## 3. Definición manual de habilidades técnicas (whitelist)
Se utiliza la misma whitelist definida manualmente en el análisis original.

In [4]:
# ... Copiar aquí la whitelist del cuaderno original ...
whitelist = {
    # Administración de Empresas
    'excel', 'power bi', 'erp', 'project management', 'sql', 'contabilidad', 'finanzas', 'marketing', 'gestión de recursos humanos', 'análisis financiero', 'auditoría', 'planificación estratégica', 'costos', 'presupuestos', 'negociación', 'comercio exterior', 'logística', 'supply chain', 'crm', 'ventas', 'administración de empresas',
    # Agroindustria
    'calidad', 'normas iso', 'producción', 'control de plagas', 'seguridad alimentaria', 'buenas prácticas agrícolas', 'procesamiento de alimentos', 'agronomía', 'fertilización', 'riego', 'manejo de cultivos', 'postcosecha', 'biotecnología', 'fitopatología', 'agroindustria', 'trazabilidad', 'certificación orgánica',
    # Ciencia de Datos
    'python', 'machine learning', 'data analysis', 'sql', 'r', 'big data', 'visualización de datos', 'estadística', 'deep learning', 'scikit-learn', 'pandas', 'numpy', 'matplotlib', 'data mining', 'análisis predictivo', 'regresión', 'clasificación', 'clustering', 'etl', 'modelado de datos', 'inteligencia artificial', 'análisis exploratorio', 'data science',
    # Computación
    'java', 'c++', 'python', 'linux', 'git', 'algoritmos', 'estructura de datos', 'sistemas operativos', 'redes', 'seguridad informática', 'programación orientada a objetos', 'desarrollo web', 'html', 'css', 'javascript', 'php', 'sql', 'c#', 'arquitectura de computadoras', 'compiladores', 'base de datos', 'ingeniería de software', 'testing', 'uml', 'scrum', 'devops',
    # Economía
    'econometría', 'estadística', 'finanzas', 'análisis de datos', 'excel', 'microeconomía', 'macroeconomía', 'modelos econométricos', 'análisis financiero', 'gestión de riesgos', 'mercados financieros', 'política económica', 'economía internacional', 'teoría de juegos', 'pronósticos', 'costos', 'presupuestos',
    # Electricidad
    'circuitos', 'electrónica', 'automatización', 'plc', 'motores eléctricos', 'instalaciones eléctricas', 'protecciones eléctricas', 'energía renovable', 'diseño eléctrico', 'subestaciones', 'mediciones eléctricas', 'generación eléctrica', 'distribución eléctrica', 'transformadores', 'sistemas eléctricos de potencia',
    # Electrónica y Automatización
    'plc', 'sensores', 'microcontroladores', 'automatización', 'circuitos', 'arduino', 'raspberry pi', 'robotica', 'sistemas embebidos', 'instrumentación', 'control automático', 'electrónica digital', 'electrónica analógica', 'scada', 'diseño de pcb', 'fpga', 'vhdl', 'proteus',
    # Física
    'simulación', 'matlab', 'python', 'análisis de datos', 'modelado', 'cálculo numérico', 'física computacional', 'teoría cuántica', 'óptica', 'mecánica clásica', 'termodinámica', 'electromagnetismo', 'análisis estadístico', 'instrumentación científica', 'latex',
    # Geología
    'gis', 'cartografía', 'sensores remotos', 'petrología', 'estratigrafía', 'geofísica', 'geotecnia', 'hidrogeología', 'mineralogía', 'mapeo geológico', 'análisis de suelos', 'modelado geológico', 'arcgis', 'qgis', 'geología estructural', 'geoquímica',
    # Ingeniería Ambiental
    'iso 14001', 'impacto ambiental', 'tratamiento de aguas', 'gis', 'normas ambientales', 'auditoría ambiental', 'gestión ambiental', 'residuos sólidos', 'remediación', 'evaluación ambiental', 'modelado ambiental', 'calidad de aire', 'sustentabilidad', 'legislación ambiental', 'educación ambiental',
    # Ingeniería Civil
    'autocad', 'sap2000', 'estructuras', 'topografía', 'cype', 'civil 3d', 'revit', 'diseño estructural', 'hormigón armado', 'cálculo estructural', 'gestión de obras', 'presupuestos', 'planificación de obras', 'geotecnia', 'hidráulica', 'vialidad', 'urbanismo',
    # Ingeniería de la Producción
    'lean manufacturing', 'six sigma', 'producción', 'erp', 'gestión de calidad', 'logística', 'supply chain', 'kaizen', 'just in time', 'mantenimiento industrial', 'planificación de la producción', 'optimización de procesos', 'simulación de procesos', 'costos industriales', 'mejora continua',
    # Ingeniería Química
    'reactores', 'procesos químicos', 'simulación', 'hysys', 'laboratorio', 'química analítica', 'balances de materia y energía', 'cromatografía', 'diseño de plantas', 'control de procesos', 'termodinámica química', 'ingeniería de procesos', 'análisis instrumental', 'química orgánica', 'química inorgánica',
    # Inteligencia Artificial
    'machine learning', 'deep learning', 'python', 'nlp', 'tensorflow', 'pytorch', 'redes neuronales', 'aprendizaje supervisado', 'aprendizaje no supervisado', 'procesamiento de lenguaje natural', 'scikit-learn', 'keras', 'inteligencia artificial', 'data mining',
    # Matemática
    'matlab', 'python', 'estadística', 'modelado', 'optimización', 'análisis numérico', 'ecuaciones diferenciales', 'probabilidad', 'teoría de números', 'álgebra lineal', 'cálculo', 'matemática aplicada', 'simulación',
    # Matemática Aplicada
    'matlab', 'python', 'estadística', 'simulación', 'optimización', 'análisis numérico', 'modelado matemático', 'ecuaciones diferenciales', 'cálculo', 'programación matemática',
    # Materiales
    'caracterización', 'microscopía', 'ensayos mecánicos', 'simulación', 'laboratorio', 'difracción de rayos x', 'metalografía', 'tratamientos térmicos', 'corrosión', 'análisis de materiales', 'propiedades mecánicas', 'microscopía electrónica',
    # Mecánica
    'solidworks', 'autocad', 'mecánica de materiales', 'catia', 'simulación', 'diseño mecánico', 'manufactura', 'elementos finitos', 'dinámica', 'termodinámica', 'cálculo estructural', 'prototipado', 'mantenimiento industrial',
    # Mecatrónica
    'plc', 'robotica', 'automatización', 'sensores', 'microcontroladores', 'sistemas embebidos', 'control automático', 'diseño mecánico', 'instrumentación', 'arduino', 'raspberry pi', 'vision artificial',
    # Petróleos
    'reservorios', 'perforación', 'simulación', 'petrel', 'ingeniería de yacimientos', 'geomecánica', 'ingeniería de perforación', 'producción petrolera', 'modelado de reservorios', 'petrofísica', 'ingeniería de producción',
    # Sistemas de Información
    'sql', 'erp', 'power bi', 'python', 'gestión de datos', 'business intelligence', 'data warehouse', 'etl', 'visualización de datos', 'base de datos', 'análisis de sistemas', 'sistemas de información',
    # Software
    'python', 'java', 'c++', 'git', 'javascript', 'typescript', 'html', 'css', 'react', 'node.js', 'desarrollo web', 'scrum', 'uml', 'testing', 'devops', 'docker', 'api', 'rest', 'c#', 'php',
    # Tecnologías de la Información
    'sql', 'cloud computing', 'python', 'linux', 'ciberseguridad', 'virtualización', 'redes', 'windows server', 'azure', 'aws', 'soporte técnico', 'seguridad informática', 'sistemas operativos', 'backup', 'firewall', 'vpn',
    # Telecomunicaciones
    'redes', 'cisco', 'fibra óptica', 'wireless', 'comunicaciones digitales', 'telefonía ip', 'microondas', 'radioenlaces', 'lte', 'routing', 'switching', 'voz sobre ip', 'protocolos de comunicación',
}
habilidades = sorted([h.lower() for h in whitelist])

## 4. Matriz término-documento (habilidad-carrera) ANTES de embeddings
Se construye la matriz de conteo de ocurrencias de cada habilidad en cada carrera, usando la whitelist.

In [5]:
carreras = list(corpus_carreras.keys())
corpus = [corpus_carreras[c].lower() for c in carreras]
vectorizer = CountVectorizer(vocabulary=habilidades, analyzer='word', ngram_range=(1, 5), lowercase=True)
X = vectorizer.fit_transform(corpus)
matriz_td = pd.DataFrame(X.T.toarray(), index=vectorizer.get_feature_names_out(), columns=carreras)
matriz_td.index.name = 'habilidad'
matriz_td.head(10)

,Administración_de_Empresas,Agroindustria,Ciencia_de_Datos,Computación,Economía,Electricidad,Electrónica_y_Automatización,Física,Geología,Ingeniería_Ambiental,...,Matemática,Matemática_Aplicada,Materiales,Mecatrónica,Mecánica,Petróleos,Sistemas_de_Información,Software,Tecnologías_de_la_Información,Telecomunicaciones
habilidad,,,,,,,,,,,,,,,,,,,,,
administración de empresas,333,15,24,21,34,6,26,5,6,50,...,0,0,14,4,4,2,169,15,34,25
agroindustria,0,72,0,0,0,0,2,0,0,2,...,0,0,0,0,0,0,0,0,0,0
agronomía,2,16,0,0,5,6,0,0,1,0,...,0,0,0,0,2,0,0,0,2,0
algoritmos,0,0,240,21,0,0,3,0,0,0,...,1,2,0,26,0,0,5,602,1,14
análisis de datos,35,5,282,133,32,1,10,6,1,31,...,9,10,20,15,7,7,290,296,88,30
análisis de materiales,0,0,0,0,0,0,0,0,0,0,...,0,0,4,0,0,0,0,1,0,0
análisis de sistemas,0,0,0,5,1,0,2,0,0,0,...,0,0,0,0,0,0,30,26,17,2
análisis de suelos,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
análisis estadístico,6,0,42,1,0,0,0,0,1,0,...,0,0,0,0,0,0,4,28,0,0


### Visualización: Ranking de habilidades por carrera (antes de embeddings)
Se muestra el top 7 de habilidades por carrera usando la matriz original.

In [6]:
def ranking_habilidades(matriz_td, top_n=7):
    for carrera in matriz_td.columns:
        top = matriz_td[carrera][matriz_td[carrera] > 0].sort_values(ascending=False).head(top_n)
        print(f'Carrera: {carrera}')
        for i, (hab, freq) in enumerate(top.items(), 1):
            print(f'  {i}. {hab} ({freq} veces)')
        print('-'*40)
ranking_habilidades(matriz_td)

Carrera: Administración_de_Empresas
  1. ventas (608 veces)
  2. calidad (599 veces)
  3. administración de empresas (333 veces)
  4. marketing (269 veces)
  5. contabilidad (219 veces)
  6. producción (209 veces)
  7. excel (180 veces)
----------------------------------------
Carrera: Agroindustria
  1. calidad (1048 veces)
  2. producción (420 veces)
  3. ventas (195 veces)
  4. riego (119 veces)
  5. seguridad alimentaria (72 veces)
  6. agroindustria (72 veces)
  7. costos (46 veces)
----------------------------------------
Carrera: Ciencia_de_Datos
  1. machine learning (1390 veces)
  2. calidad (564 veces)
  3. python (466 veces)
  4. producción (385 veces)
  5. ventas (337 veces)
  6. aws (337 veces)
  7. sql (304 veces)
----------------------------------------
Carrera: Computación
  1. etl (1514 veces)
  2. sql (1347 veces)
  3. calidad (848 veces)
  4. python (368 veces)
  5. producción (337 veces)
  6. aws (333 veces)
  7. azure (283 veces)
-----------------------------------

## 5. Agrupación de términos usando embeddings
Se generan embeddings para cada término de la whitelist y se agrupan términos similares usando clustering.

**Aquí se evidencia el antes y el después:**
- Se visualizan los grupos de términos similares.
- Se muestra qué términos conforman cada grupo (embedding).

In [9]:
# Cargar modelo de embeddings
modelo = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = modelo.encode(habilidades)

# Calcular matriz de similitud coseno
sim_matrix = cosine_similarity(embeddings)

# Clustering simple: agrupamos términos con similitud > 0.75
from sklearn.cluster import AgglomerativeClustering
clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=0.5, metric='cosine', linkage='average')
labels = clustering.fit_predict(embeddings)

# Mostrar los grupos de términos
from collections import defaultdict
grupos = defaultdict(list)
for term, label in zip(habilidades, labels):
    grupos[label].append(term)
for label, terms in grupos.items():
    print(f'Grupo {label+1}: {terms}')

Grupo 94: ['administración de empresas']
Grupo 45: ['agroindustria', 'agronomía']
Grupo 19: ['algoritmos', 'hormigón armado']
Grupo 39: ['análisis de datos', 'base de datos', 'estructura de datos', 'gestión de datos', 'modelado de datos', 'visualización de datos']
Grupo 28: ['análisis de materiales', 'análisis de suelos', 'mecánica de materiales']
Grupo 77: ['análisis de sistemas', 'sistemas embebidos', 'sistemas operativos']
Grupo 111: ['análisis estadístico', 'análisis exploratorio', 'análisis numérico', 'análisis predictivo']
Grupo 9: ['análisis financiero', 'finanzas', 'mercados financieros']
Grupo 32: ['análisis instrumental', 'instrumentación', 'instrumentación científica']
Grupo 116: ['api']
Grupo 3: ['aprendizaje no supervisado', 'aprendizaje supervisado', 'caracterización', 'clasificación', 'negociación', 'optimización', 'remediación', 'simulación', 'soporte técnico']
Grupo 136: ['arcgis']
Grupo 27: ['arduino', 'linux', 'raspberry pi']
Grupo 13: ['arquitectura de computadoras'

## 6. Matriz término-documento DESPUÉS de embeddings
Se agrupan los conteos de términos similares (por grupo de embedding) y se reconstruye la matriz habilidad-carrera.
Esto reduce la dimensionalidad y permite ver el impacto de los embeddings.

In [11]:
# Crear nueva matriz agrupando por grupo de embedding
matriz_td_emb = pd.DataFrame(0, index=grupos.keys(), columns=carreras)
for label, terms in grupos.items():
    matriz_td_emb.loc[label] = matriz_td.loc[terms].sum()
# Renombrar filas con una muestra de términos representativos
matriz_td_emb.index = [', '.join(grupos[label][:3]) + (' ...' if len(grupos[label])>3 else '') for label in grupos.keys()]
matriz_td_emb.head(10)

,Administración_de_Empresas,Agroindustria,Ciencia_de_Datos,Computación,Economía,Electricidad,Electrónica_y_Automatización,Física,Geología,Ingeniería_Ambiental,...,Matemática,Matemática_Aplicada,Materiales,Mecatrónica,Mecánica,Petróleos,Sistemas_de_Información,Software,Tecnologías_de_la_Información,Telecomunicaciones
administración de empresas,333,15,24,21,34,6,26,5,6,50,...,0,0,14,4,4,2,169,15,34,25
"agroindustria, agronomía",2,88,0,0,5,6,2,0,1,2,...,0,0,0,0,2,0,0,0,2,0
"algoritmos, hormigón armado",0,0,240,21,0,0,3,0,0,0,...,1,2,0,26,0,0,5,602,1,14
"análisis de datos, base de datos, estructura de datos ...",91,11,528,571,45,4,22,18,11,57,...,17,19,34,41,11,9,782,1025,244,46
"análisis de materiales, análisis de suelos, mecánica de materiales",0,0,0,0,0,0,0,0,0,0,...,0,0,4,0,2,0,0,1,0,0
"análisis de sistemas, sistemas embebidos, sistemas operativos",32,1,7,50,2,14,9,3,2,5,...,1,1,1,11,3,1,75,400,189,63
"análisis estadístico, análisis exploratorio, análisis numérico ...",8,0,64,1,0,0,0,2,1,1,...,5,2,0,0,0,0,5,43,8,0
"análisis financiero, finanzas, mercados financieros",114,10,30,44,101,6,90,16,10,36,...,2,0,17,80,5,19,1298,210,47,13
"análisis instrumental, instrumentación, instrumentación científica",1,0,1,0,0,152,59,0,0,1,...,0,0,9,20,15,14,6,17,6,17
api,3,0,92,152,0,0,10,0,0,3,...,4,1,0,42,15,11,314,3223,62,22


### Visualización: Ranking de habilidades por carrera (después de embeddings)
Se muestra el top 7 de grupos de habilidades por carrera usando la matriz reducida.

In [12]:
ranking_habilidades(matriz_td_emb)

Carrera: Administración_de_Empresas
  1. calidad, cálculo, cálculo estructural ... (630 veces)
  2. ventas (608 veces)
  3. administración de empresas (333 veces)
  4. marketing (269 veces)
  5. contabilidad, sustentabilidad (245 veces)
  6. ingeniería de producción, planificación de la producción, producción ... (211 veces)
  7. excel, matlab (180 veces)
----------------------------------------
Carrera: Agroindustria
  1. calidad, cálculo, cálculo estructural ... (1068 veces)
  2. ingeniería de producción, planificación de la producción, producción ... (420 veces)
  3. ventas (195 veces)
  4. gestión de riesgos, riego (119 veces)
  5. agroindustria, agronomía (88 veces)
  6. seguridad alimentaria, seguridad informática, sistemas de información (72 veces)
  7. aprendizaje no supervisado, aprendizaje supervisado, caracterización ... (61 veces)
----------------------------------------
Carrera: Ciencia_de_Datos
  1. big data, business intelligence, clustering ... (1775 veces)
  2. calidad

## 7. Comparación visual antes y después de embeddings
Se puede comparar el número de dimensiones (habilidades) y los rankings para ver el efecto de los embeddings.

- Número de términos antes: 
- Número de grupos después: 

In [13]:
print(f'Número de términos antes: {len(habilidades)}')
print(f'Número de grupos después: {len(grupos)}')

Número de términos antes: 288
Número de grupos después: 156
